TEAM, VENUE AND PLAYER DIMENSION UPSERT

In [0]:
-- TEAMS 
MERGE INTO dw.dim_team
USING (
  SELECT DISTINCT
    LOWER(TRIM(team)) AS team_key,
    team              AS team_name
  FROM (
    SELECT first_team AS team FROM stg.match_table
    UNION ALL
    SELECT second_team FROM stg.match_table
    UNION ALL
    SELECT toss_winner FROM stg.match_table
    UNION ALL
    SELECT winner FROM stg.match_table
  ) t
  WHERE team IS NOT NULL
) s
ON dw.dim_team.team_key = s.team_key
WHEN MATCHED THEN UPDATE
  SET team_name = s.team_name
WHEN NOT MATCHED THEN INSERT (team_key, team_name)
VALUES (s.team_key, s.team_name);



-- VENUES
DROP TABLE IF EXISTS stg_venues_tmp;

CREATE TEMP TABLE stg_venues_tmp AS
SELECT DISTINCT
  LOWER(TRIM(COALESCE(venue,''))) || '|' ||
  LOWER(TRIM(COALESCE(city,'')))   AS venue_key,
  venue    AS venue_name,
  city
FROM stg.match_table;

-- MERGE with the temp table
MERGE INTO dw.dim_venue
USING stg_venues_tmp s
ON dw.dim_venue.venue_key = s.venue_key
WHEN MATCHED THEN UPDATE
  SET venue_name = s.venue_name,
      city       = s.city
WHEN NOT MATCHED THEN INSERT (venue_key, venue_name, city)
VALUES (s.venue_key, s.venue_name, s.city);



-- PLAYERS (from staging roster and delivery names)
MERGE INTO dw.dim_player
USING (
  SELECT DISTINCT
    LOWER(TRIM(p.player_name)) || '|' || LOWER(TRIM(t.team_key)) AS player_key,
    p.player_name AS full_name,
    t.team_id
  FROM stg.player_table p
  JOIN dw.dim_team t
    ON LOWER(TRIM(p.country)) = t.team_key   -- roster country → team_key
  WHERE p.player_name IS NOT NULL
) s
ON dw.dim_player.player_key = s.player_key
WHEN MATCHED THEN UPDATE
  SET full_name = s.full_name,
      team_id   = s.team_id
WHEN NOT MATCHED THEN INSERT (player_key, full_name, team_id)
VALUES (s.player_key, s.full_name, s.team_id);

DATE DIMENSION UPSERT

In [0]:
INSERT INTO dw.dim_date (date_key, full_date, year, month, day, week, quarter, season)
SELECT
  CAST(TO_CHAR(d, 'YYYYMMDD') AS INT)      AS date_key,
  d                                        AS full_date,
  EXTRACT(year FROM d)::int                AS year,
  EXTRACT(month FROM d)::int               AS month,
  EXTRACT(day FROM d)::int                 AS day,
  EXTRACT(week FROM d)::int                AS week,
  EXTRACT(quarter FROM d)::int             AS quarter,
  TO_CHAR(d, 'YYYY')                       AS season
FROM (
    SELECT DISTINCT event_date::date AS d
    FROM stg.match_table
    WHERE event_date IS NOT NULL
) dates
LEFT JOIN dw.dim_date dd ON dd.full_date = dates.d
WHERE dd.full_date IS NULL;

CORE MATCH DIMENSION UPSERT

In [0]:
MERGE INTO dw.dim_match
USING (
  SELECT
    m.match_type_number                         AS match_id,
    m.event_name,
    m.match_type,
    m.match_stage,
    m.event_date::date                          AS event_date,
    CAST(TO_CHAR(m.event_date, 'YYYYMMDD') AS INT) AS date_key,
    m.season,
    m.team_type,
    m.overs,
    v.venue_id,
    m.gender,
    t1.team_id AS first_team_id,
    t2.team_id AS second_team_id,
    m.match_result,
    tw.team_id AS toss_winner_id,
    ww.team_id AS winner_team_id,
    m.toss_decision,
    m.stg_file_name,
    m.stg_file_hashkey,
    m.ingest_date
  FROM stg.match_table m
  LEFT JOIN dw.dim_team t1 ON t1.team_key = LOWER(TRIM(m.first_team))
  LEFT JOIN dw.dim_team t2 ON t2.team_key = LOWER(TRIM(m.second_team))
  LEFT JOIN dw.dim_team tw ON tw.team_key = LOWER(TRIM(m.toss_winner))
  LEFT JOIN dw.dim_team ww ON ww.team_key = LOWER(TRIM(m.winner))
  LEFT JOIN dw.dim_venue v
    ON v.venue_key = LOWER(TRIM(m.venue)) || '|' || LOWER(TRIM(m.city))
) s
ON dw.dim_match.match_id = s.match_id
WHEN MATCHED THEN UPDATE SET
  event_name       = s.event_name,
  match_type       = s.match_type,
  match_stage      = s.match_stage,
  event_date       = s.event_date,
  date_key         = s.date_key,
  season           = s.season,
  team_type        = s.team_type,
  overs            = s.overs,
  venue_id         = s.venue_id,
  gender           = s.gender,
  first_team_id    = s.first_team_id,
  second_team_id   = s.second_team_id,
  match_result     = s.match_result,
  winner_team_id   = s.winner_team_id,
  toss_winner_id   = s.toss_winner_id,
  toss_decision    = s.toss_decision,
  stg_file_name    = s.stg_file_name,
  stg_file_hashkey = s.stg_file_hashkey,
  ingest_date      = s.ingest_date
WHEN NOT MATCHED THEN INSERT (
  match_id, event_name, match_type, match_stage, event_date, date_key, season,
  team_type, overs, venue_id, gender, first_team_id, second_team_id, match_result,
  winner_team_id, toss_winner_id, toss_decision, stg_file_name, stg_file_hashkey, ingest_date
)
VALUES (
  s.match_id, s.event_name, s.match_type, s.match_stage, s.event_date, s.date_key, s.season,
  s.team_type, s.overs, s.venue_id, s.gender, s.first_team_id, s.second_team_id, s.match_result,
  s.winner_team_id, s.toss_winner_id, s.toss_decision, s.stg_file_name, s.stg_file_hashkey, s.ingest_date
);